In [ ]:
#импорт необходимых библиотек; заранее в терминале выполнить команду pip install requests pandas dotenv
import requests
import os
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

In [6]:
#присваивание токена. Сам токен редактируется в .env
wb_api_token=os.getenv('wb_api_token')

In [9]:
#Устанавливается пользователем
date_from = '2025-08-08'
date_to = '2025-08-09'
searchTextes = [
    "школьный рюкзак",
    "школьный рюкзак для девочек 5 класс",
    "школьный рюкзак для девочек"
]
nmId=225056490

In [ ]:
# ===== АНАЛИТИКА =====
# ==== Заказы и позиции по поисковым запросам товара ====
# === ТОКЕН ===
# ЗАМЕНИ НА СВОЙ (из кабинета продавца)
# == ПАРАМЕТРЫ ==
nmId = int(os.getenv('nmId'))
#Дата устанавливается пользователем, разница в датах не более 7 дней
date_from = '2025-08-09'
date_to = '2025-08-15'
#Кластеры устанавливаются пользователем
searchTexts = [
    "школьный рюкзак",
    "школьный рюкзак для девочек 5 класс",
    "школьный рюкзак для девочек"
]

params = {
    "period": {"start": date_from, "end": date_to},
    "nmId": nmId,
    "searchTexts": searchTexts
}

headers = {
    'Authorization': wb_api_token,
    'Content-Type': 'application/json'
}

url = 'https://seller-analytics-api.wildberries.ru/api/v2/search-report/product/orders'

# === ЗАПРОС ===
res = requests.post(url, headers=headers, json=params)

# === ОБРАБОТКА ОТВЕТА ===
if res.status_code != 200:
    print(f"❌ Ошибка API: {res.status_code}")
    print("Текст ошибки:", res.text)
    exit()

# Парсим JSON
try:
    data_dict = res.json()
except Exception as e:
    print("❌ Не удалось распарсить JSON:", e)
    exit()

# Извлекаем 'items' из 'data'
items = data_dict.get('data', {}).get('items', [])

if not items:
    print("⚠️ Нет данных в 'items'. Ответ:", data_dict)
    exit()

# === 5. СОБИРАЕМ ДАННЫЕ В СПИСОК СЛОВАРЕЙ ===
rows = []
for item in items:
    text = item.get('text', 'неизвестно')
    frequency = item.get('frequency', 0)
    for day in item.get('dateItems', []):
        rows.append({
            'Поисковый запрос': text,
            'Частота': frequency,
            'Дата': day.get('dt', ''),
            'Средняя позиция': day.get('avgPosition', 0),
            'Заказы': day.get('orders', 0)
        })

# === СОЗДАЁМ DataFrame ===
df = pd.DataFrame(rows)

# Проверим: сколько строк и столбцов
print("✅ DataFrame создан:")
print(df.head())
print("\nСтруктура данных:")
print(df.dtypes)

# === СОХРАНЯЕМ В CSV С ПРАВИЛЬНЫМ РАЗДЕЛИТЕЛЕМ ===
try:
    df.to_csv(
        'wb_data.csv',
        index=False,
        encoding='utf-8-sig',
        sep=';'  # ← Используем точку с запятой как разделитель (лучше для Excel)
    )
    print("✅ Файл 'wb_data_status.csv' сохранён с разделителем ';'")
    print("Открой его в Excel — столбцы будут разделены.")
except Exception as e:
    print(f"❌ Ошибка при сохранении: {e}")